In [1]:
import numpy as np
import torch

from s3ts.helper_functions import *

In [3]:
dm = load_dmdataset("WISDM", "./datasets/", 0.1, 128, 8, 48, 1, True, 48, 250, [30, 31, 32, 33, 34, 35], True, 2, 1)

Loaded dataset WISDM with a total of 1096480 observations for window size 48
Computing medoids...
Computing dissimilarity frames...
Loading cached dissimilarity frames if available...
Using 443636 observations for training and 154375 observations for validation and test


In [6]:
from s3ts.api.nets.methods import create_model_from_DM, train_model

model = create_model_from_DM(dm, "img", "cnn_gap", "mlp", "cls", "test", 20, 64, 1, 0.001, voting={"n": 1, "rho": 0.1})

Input shape:  torch.Size([1, 12, 48, 48])
Latent shape:  torch.Size([1, 40, 5])
[tensor(200), 64, 6]


In [7]:
train_model(dm, model, 3, {
    "default_root_dir": "training",
    "accelerator": "auto",
    "seed": 42
})

Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type                      | Params
---------------------------------------------------------
0 | encoder    | CNN_GAP_IMG               | 18.3 K
1 | decoder    | MultiLayerPerceptron      | 13.3 K
2 | flatten    | Flatten                   | 0     
3 | softmax    | Softmax                   | 0     
4 | train_cm   | MulticlassConfusionMatrix | 0     
5 | val_cm     | MulticlassConfusionMatrix | 0     
6 | val_auroc  | MulticlassAUROC           | 0     
7 | test_cm    | MulticlassConfusionMatrix | 0     
8 | test_auroc | MulticlassAUROC           | 0     
---------------------------------------------------------
31.5 K    Trainable params
0         Non-trainable params
31.5 K    Total params
0.126     Total estimated model params size (MB)


Epoch 2: 100%|██████████| 3466/3466 [07:59<00:00,  7.23it/s, v_num=18, train_loss_step=0.023, val_loss=0.668, val_auroc=0.935, val_re=0.782, train_loss_epoch=0.0813, train_re=0.969] 

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 3466/3466 [07:59<00:00,  7.23it/s, v_num=18, train_loss_step=0.023, val_loss=0.668, val_auroc=0.935, val_re=0.782, train_loss_epoch=0.0813, train_re=0.969]
Input shape:  torch.Size([1, 12, 48, 48])
Latent shape:  torch.Size([1, 40, 5])
[tensor(200), 64, 6]
Validation DataLoader 0: 100%|██████████| 1207/1207 [01:00<00:00, 19.83it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_auroc         │    0.9592697620391846     │
│          val_f1           │    0.8081207871437073     │
│         val_loss          │    0.5647413730621338     │
│          val_pr           │     0.850165843963623     │
│          val_re           │    0.8028659820556641     │
└───────────────────────────┴───────────────────────────┘

(WrapperModel(
   (encoder): CNN_GAP_IMG(
     (cnn_0): Sequential(
       (0): Conv2d(12, 10, kernel_size=(7, 7), stride=(1, 1), padding=same)
       (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU()
       (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     )
     (cnn_1): Sequential(
       (0): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1), padding=same)
       (1): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU()
       (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     )
     (cnn_2): Sequential(
       (0): Conv2d(20, 40, kernel_size=(3, 3), stride=(1, 1), padding=valid)
       (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU()
       (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     )
   )
   (decoder): MultiLayerPercept

In [7]:
import torchmetrics as tm

In [8]:
acc_online = tm.Accuracy(task="multiclass", num_classes=6, average="macro")
cm_online = tm.ConfusionMatrix(task="multiclass", num_classes=6)
dataloader = dm.val_dataloader()

In [9]:
model.eval()
for batch_idx, data in enumerate(dataloader):
    preds = model.logits(data["series"])
    acc_online(preds, data["label"])
    cl = torch.argmax(preds, dim=1)
    cm_online(cl, data["label"])


In [10]:
acc_online.compute()

tensor(0.8273)

In [11]:
cm = cm_online.compute()

In [12]:
TP = cm.diag()
FP = cm.sum(0) - TP
FN = cm.sum(1) - TP
TN = torch.empty(cm.shape[0])
for i in range(cm.shape[0]):
    TN[i] = cm[:i,:i].sum() + cm[:i,i:].sum() + cm[i:,:i].sum() + cm[i:,i:].sum()

In [13]:
accuracy = (TP + TN)/(TP + TN + FP + FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)

In [14]:
accuracy

tensor([0.9405, 0.9863, 0.9907, 0.9917, 0.9052, 0.9293])

In [15]:
recall

tensor([0.6781, 0.9370, 0.8843, 0.8330, 0.8169, 0.8146])

In [16]:
recall.mean()

tensor(0.8273)